# Punto 1

Selección de autores y libros: Elige tres autores cuyos trabajos estén disponibles en el Proyecto Gutenberg. Para cada uno de estos autores, selecciona y descarga cuidadosamente un mínimo de tres libros, sumando al menos nueve libros en total.
Entrenamiento de embeddings: Utiliza estas obras literarias seleccionadas para entrenar word embeddings utilizando la biblioteca GENSIM y el modelo Word2Vec.

•	Prepara el conjunto de datos para el entrenamiento utilizando pasos adecuados de preprocesamiento de texto.
•	Prueba diferentes dimensionalidades de los embeddings (al menos 3) y guárdalos en disco utilizando los métodos apropiados de GENSIM con el siguiente formato de nombre de archivo:

Books_<size_1>_<group_code>

Books_<size_2>_<group_code>

Books_<size_3>_<group_code>

Importación de bibliotecas y dependencias a usar en el ejercicio

In [1]:
import os
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
import string
from sklearn.calibration import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd
from gensim.models import KeyedVectors

Se implementa una función que toma como entrada un texto de un libro de Gutenberg y elimina toda la información no relevante (introducciones, metadatos, nombres de autores y secciones innecesarias) para devolver solo el contenido principal del libro. Esto facilita el uso posterior del texto para tareas de procesamiento de lenguaje natural, ya que solo quedará el contenido que realmente importa.

In [2]:
def clean_gutenberg_text(text):
    # Eliminar el contenido antes de "*** START OF THE PROJECT GUTENBERG EBOOK ***"
    text = re.sub(r'^.*?\*\*\* START OF THE PROJECT GUTENBERG EBOOK.*?\*\*\*', '', text, flags=re.DOTALL)
    # Eliminar el contenido después de "*** END OF THE PROJECT GUTENBERG EBOOK ***"
    text = re.sub(r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK.*?$', '', text, flags=re.DOTALL)
    # Eliminar las menciones de autores específicos
    authors = r'William Oberfield|James Branch Cabell|Wilhelm Raabe'
    text = re.sub(r'(?i)\b(?:' + authors + r')\b.*?(?=\n)', '', text, flags=re.DOTALL)
    text = re.sub(r'(Produced by.*?Distributed Proofreaders|Title:.*?Author:.*?Release date:.*?Language:.*?Credits:.*?Project Gutenberg Distributed Proofreaders)', '', text, flags=re.DOTALL)
    text = re.sub(r'(CONTENTS.*?THE AFTERWORD|BIBLIOGRAPHY|INDEX)', '', text, flags=re.DOTALL)
    text = re.sub(r'\n+', '\n', text)
    text = text.strip()
    
    return text

Se implementa una función para el preprocesaiento del texto, que toma como entrada un texto y lo preprocesa eliminando metadatos, números, puntuación y palabras vacías. Convierte el texto a minúsculas, lo tokeniza y aplica lematización para convertir las palabras a sus formas base. Este texto procesado es adecuado para ser utilizado en modelos de NLP para el manejo de embeddings que se quiere realizar.

In [3]:
def preprocess_text(text):
    # Limpiar el texto usando la función clean_gutenberg_text para eliminar secciones no deseadas
    text = clean_gutenberg_text(text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizar el texto, es decir, dividirlo en palabras individuales
    tokens = word_tokenize(text)
    # Lematizar los tokens, eliminando las palabras que están en la lista de stopwords (palabras comunes como 'the', 'is', 'in')
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    return tokens

Se implementa una función que recorre todos los archivos de texto (.txt) en un directorio, los abre, los lee, los preprocesa usando la función preprocess_text, y almacena las palabras procesadas (tokens) de cada archivo en una lista llamada corpus. El resultado es un conjunto de textos preprocesados listos para ser usados en modelos de procesamiento de lenguaje natural (NLP), donde cada archivo de texto se ha convertido en una lista de palabras procesadas y normalizadas.

In [4]:
def load_books_from_directory(directory):
    corpus = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                # Abre el archivo en modo lectura con codificación UTF-8
                with open(file_path, 'r', encoding='utf-8') as f:
                    global text
                    text = f.read()
                    # Preprocesa el texto (limpieza, tokenización, etc.) y devuelve los tokens
                    tokens = preprocess_text(text)
                    # Agrega los tokens procesados al corpus
                    corpus.append(tokens)
    return corpus

Esta implementación, entrena modelos de Word2Vec con embeddings de 300, 400 y 500 dimensiones a partir de un corpus de textos preprocesados. El corpus se carga desde un directorio, y luego, en cada iteración, se entrena un modelo con un tamaño de embedding específico. Los modelos entrenados se guardan en archivos .model y se imprime una confirmación en la consola.

In [5]:
directory = 'books' 
corpus = load_books_from_directory(directory)

# Lista de tamaños de embedding que se usarán para entrenar los modelos
embedding_sizes = [300, 400, 500]
group_code = "G4"

for size in embedding_sizes:
    # Crea un modelo Word2Vec con el corpus cargado
    # vector_size: tamaño del vector de embedding
    # window: número de palabras a considerar en el contexto a ambos lados
    # min_count: ignora palabras que aparecen menos de 5 veces
    # workers: número de núcleos de CPU usados para el entrenamiento
    model = Word2Vec(sentences=corpus, vector_size=size, window=15, min_count=5, workers=4)
    
    model.save(f'Books_{size}_{group_code}.model')
    print(f"Saved model with size {size} as 'Books_{size}_{group_code}.model'")

print("Training completed.")

Saved model with size 300 as 'Books_300_G4.model'
Saved model with size 400 as 'Books_400_G4.model'
Saved model with size 500 as 'Books_500_G4.model'
Training completed.
